In [178]:
def generate_reads(sequence, min_subseq_len, max_subseq_len, min_overlap, max_overlap,seed=None):
    import random

    random.seed(seed)
    reads = []
    start = 0
    end = random.randint(min_subseq_len,max_subseq_len)
    reads += [sequence[start:end]]
    while end < len(sequence):
        start = random.randint(end-max_overlap,end-min_overlap)
        if (len(sequence) - start)/max_subseq_len < 2:
            if (len(sequence) - start)/max_subseq_len < 1:
                end = len(sequence)
            else:
                a = 0
                while (len(sequence) - start)/(min_subseq_len+a) > 2: a+=1
                end = random.randint(start+min_subseq_len+a,start+max_subseq_len) 
        else: end = random.randint(start+min_subseq_len,start+max_subseq_len) 
        reads += [sequence[start:end]]
    return reads

def generate_genome_sequence(n,seed=None):
    import random
    
    random.seed(seed)
    nucleotides = {1:'A',2:'C',3:'G',4:'T'}
    seq = ''
    for i in range(n):
        seq += nucleotides[random.randint(1,4)]
    return seq

def remove_containments(reads):
    i = 0
    r = set()
    m = max([len(r) for r in reads])
    b = False
    while i < len(reads):
        if len(reads[i]) == m: 
            r.add(reads[i])
            i+=1
            continue
        for r_ in r:
            if reads[i] in r_: 
                b = True
                break
        if b: 
            b = False
            i+=1
            continue
        r.add(reads[i])
        i+=1
    return list(r)

# Sequitur

In [179]:
'''
DESCRIPTION
INPUT
OUTPUT
'''
class Stalk:
    def __init__(self,stalk):
        self.stalk = stalk
        if len(self.stalk) == 0: self.stalk = '$'

    def __repr__(self): return self.stalk

    def __eq__(self,other): return self[0] == other[0]

    def __hash__(self): return hash(self[0])

    def __getitem__(self,index): return self.stalk[index]

    def __len__(self):
        if self.stalk == '$' or self.stalk == '^': return 0
        return len(self.stalk)
    
    def __str__(self):
        if self.stalk =='$': return ''
        else: return self.stalk

    '''
    DESCRIPTION
    INPUT
    OUTPUT
    '''
    def common_substring(self,other):
        i = 0
        substr = ''
        if type(other) == str: other = Stalk(other)
        while i < min(len(self),len(other)) and self[i] == other[i]:
            substr += self[i]
            i += 1
        return Stalk(substr), Stalk(self[i:]), Stalk(other[i:])

In [180]:
'''
DESCRIPTION
INPUT
OUTPUT
'''
class Leaf:
    def __init__(self,left,right=''):
        if len(left) == 0:
            self.left = left
            self.right = 1
        else:
            self.left = left
            self.right = Leaf(right)
    
    def __repr__(self): return str(self.right)

    def __eq__(self,other): return self[0] == other[0]

    def __hash__(self): return hash(self.left)

    def __getitem__(self,index): return self.left[index]

    def __len__(self):
        if self.left == '$': return 0
        return len(self.left)
    
    def __is_shallow__(self): return True

    def reads(self): return set()

In [181]:
'''
DESCRIPTION
INPUT
OUTPUT
    '''
class Branch:
    def __init__(self):
        self.b = {}
        self.s = {}

    def __repr__(self): return repr(self.b)

    def __str__(self):
        s = '' 
        for i in range(len(list(self.b.values()))-1):s+=str(list(self.b.values())[i])+'\n'
        return s+str(list(self.b.values())[-1])
    
    def __getitem__(self,index):
        if type(index) == str: return self.b[Stalk(index)]
        return self.b[index]

    '''
    DESCRIPTION
    INPUT
    OUTPUT
    '''
    def __is_shallow__(self):
        for a in self.b.values():
            if type(a) == Branch: return False
        return True

    '''
    DESCRIPTION
    INPUT
    OUTPUT
    '''
    def __traverse__(self,context):
        b = self[context[0]]
        s = self.s[context[0]]
        context = context[len(s[0]):]
        while len(context) > 0 and len(b) > 1:
            s = b.s[context[0]]
            b = b[context[0]]
            context = context[len(s[0]):]
        return b
    
    def __setitem__(self,index,value):
        if type(index) == str: self.b[Stalk(index)] = value
        else: self.b[index] = value

    def __contains__(self,other): 
        if type(other) == str: return Stalk(other) in self.b
        return other in self.b

    def __len__(self): return len(self.b)

    def pop(self,index): return self.b.pop(index)

    '''
    DESCRIPTION
        adds a suffix to the trie
    INPUT
        stalk | a Stalk() which is a common substring of every read up to this point and beyond
        reads | a set of reads which have with the same common substring up to this point
    '''
    def add(self,stalk,reads):
        if stalk in self:
            if not len(stalk):
                self[stalk].right+=1
                self.s[stalk][1].update(reads)
                return
            if type(self[stalk]) == Leaf:
                branch = Branch()
                l1 = self.pop(stalk)
                stalk_ = list(self.s.pop(stalk))
                stalk_[0],l1.left,l2 = stalk_[0].common_substring(stalk)
                branch.add(l1.left,stalk_[1].copy())
                stalk_[1].update(reads)
                branch.add(l2,reads)
                stalk_ = tuple(stalk_)
                self[stalk_[0]] = branch
                self.s[stalk_[0]] = stalk_
            else:
                stalk_ = list(self.s.pop(stalk))
                branch = self.pop(stalk)
                stalk_[0],bstalk,stalk = stalk_[0].common_substring(stalk)
                if len(bstalk):
                    br = Branch()
                    br[bstalk] = branch 
                    br.s[bstalk] = (bstalk,stalk_[1].copy())
                    br.add(stalk,reads)
                    self[stalk_[0]] = br
                else: 
                    branch.add(stalk,reads)
                stalk_[1].update(reads)
                stalk_ = tuple(stalk_)
                if not len(bstalk): self[stalk_[0]] = branch
                self.s[stalk_[0]] = stalk_
        else:
            if type(stalk) == str: stalk = Stalk(stalk)
            self.s[stalk] = (stalk,reads)
            self[stalk] = Leaf(stalk)

In [246]:
'''
DESCRIPTION
    an object which constructs a suffix trie out of fragments of a sequence and can traverse 
    the trie to resconstruct some target sequence
INPUT
    reads | a list of strings which overlap and are fragments of a longer sequence
'''
class Sequitur:
    def __init__(self,reads,correct_sequence=None,k_min=3,**kwargs):
        self.correct_sequence = correct_sequence
        self.branch = Branch()
        self.reads = reads
        self.k_min = k_min
        for read in reads:
            for i in range(len(read)):
                if len(read[i:]) < self.k_min: continue 
                self.branch.add(Stalk(read[i:]),{read})
        if "len" in kwargs or ("assemble" in kwargs and kwargs["assemble"]): self.phase1(**kwargs)

    def phase1(self,**kwargs): 
        if "k_min_add" not in kwargs: kwargs["k_min_add"] = 0   
        extensions = {}
        stalks = self.branch.b.keys()
        for stalk in stalks: self.longest_common_substring(self.branch,stalk,[stalk.stalk],extensions)
        k_max = max(extensions.keys())
        i = 0
        overlaps = {}
        if "len" in kwargs and kwargs["len"] == len(self.reads):
            if self.k_min + kwargs["k_min_add"] < k_max: kwargs["k_min_add"] += 1
            else:
                # if self.phase2(self,extensions):
                    # try to extend read with an extension recursively
                    # if extension runs into a situation where there is a read that cannot be added, rollback
                # else:
                self.sequence = self.reads
                return
        else:
            kwargs["k_min_add"] = 0 
            kwargs["len"] = len(self.reads)
        for read in self.reads:
            while min(k_max,len(read)-1) - i > self.k_min + kwargs["k_min_add"]:
                if read[:min(k_max,len(read)-1)-i] in extensions[min(k_max,len(read)-1)-i]:
                    if len(extensions[min(k_max,len(read)-1)-i][read[:min(k_max,len(read)-1)-i]]['endswith']) > 1: i += 1
                    else: break
                else: i+=1
            if (read[:min(k_max,len(read)-1)-i] in extensions[min(k_max,len(read)-1)-i] and len(extensions[min(k_max,len(read)-1)-i][read[:min(k_max,len(read)-1)-i]]['endswith']) > 1)\
            or read[:min(k_max,len(read)-1)-i] not in extensions[min(k_max,len(read)-1)-i] or read not in extensions[min(k_max,len(read)-1)-i][read[:min(k_max,len(read)-1)-i]]['is_in']:
                i = 0
                continue
            if list(extensions[min(k_max,len(read)-1)-i][read[:min(k_max,len(read)-1)-i]]['endswith'])[0] not in overlaps: overlaps[list(extensions[min(k_max,len(read)-1)-i][read[:min(k_max,len(read)-1)-i]]['endswith'])[0]] = (read[:min(k_max,len(read)-1)-i],read,read[min(k_max,len(read)-1)-i:])
            else: overlaps[list(extensions[min(k_max,len(read)-1)-i][read[:min(k_max,len(read)-1)-i]]['endswith'])[0]] = ('','','')
            i = 0
        if len(overlaps):
            overlaps = list(overlaps.items())
            overlaps.sort(key=lambda e: len(e[1][0]),reverse=True)
            overlaps = dict(overlaps)
            key = list(overlaps.keys())[0]
            seq = key
            self.reads.remove(key)
            while key in overlaps:
                if len(overlaps[key][0]) < sum(len(o[0]) for o in overlaps.values())/len(overlaps): break
                seq += overlaps[key][2]
                key = overlaps[key][1]
                if key not in self.reads or not len(key): break
                self.reads.remove(key)
            self.reads += [seq]
        if len(self.reads) == 1: 
            self.sequence = self.reads[0]
            return
        else:
            self.__init__(self.reads,self.correct_sequence,self.k_min,**kwargs)
        
    def phase2(self,extensions):
        pass
        
    def longest_common_substring(self,branch,stalk,substring,extensions):
        if branch.__is_shallow__():
            if len(branch.s[stalk][1]) > 1:
                if len(''.join(substring)) not in extensions: extensions[len(''.join(substring))] = {}
                if ''.join(substring) not in extensions[len(''.join(substring))]: extensions[len(''.join(substring))][''.join(substring)] = {'endswith':set(),'is_in':set()}
                for read in branch.s[stalk][1]:
                    if read.endswith(''.join(substring)): extensions[len(''.join(substring))][''.join(substring)]['endswith'].add(read)
                    else: extensions[len(''.join(substring))][''.join(substring)]['is_in'].add(read)
            return extensions
        if type(branch.__traverse__(stalk.stalk)) is Leaf:
            if len(branch.s[stalk][1]) > 1:
                if len(''.join(substring)) not in extensions: extensions[len(''.join(substring))] = {}
                if ''.join(substring) not in extensions[len(''.join(substring))]: extensions[len(''.join(substring))][''.join(substring)] = {'endswith':set(),'is_in':set()}
                for read in branch.s[stalk][1]:
                    if read.endswith(''.join(substring)): extensions[len(''.join(substring))][''.join(substring)]['endswith'].add(read)
                    else: extensions[len(''.join(substring))][''.join(substring)]['is_in'].add(read)
            return extensions
        for c in branch.__traverse__(stalk.stalk).b:
            if c.stalk != '$': extensions = self.longest_common_substring(branch.__traverse__(stalk.stalk),c,substring+[c.stalk],extensions)
            else: 
                if len(branch.s[stalk][1]) > 1:
                    if len(''.join(substring)) not in extensions: extensions[len(''.join(substring))] = {}
                    if ''.join(substring) not in extensions[len(''.join(substring))]: extensions[len(''.join(substring))][''.join(substring)] = {'endswith':set(),'is_in':set()}
                    for read in branch.s[stalk][1]:
                        if read.endswith(''.join(substring)): extensions[len(''.join(substring))][''.join(substring)]['endswith'].add(read)
                        else: extensions[len(''.join(substring))][''.join(substring)]['is_in'].add(read)
        return extensions

In [240]:
sequence = 'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'
reads = ['betty_bought_butter_th',
                        'tter_the_butter_was_',
                              'he_butter_was_bitter_',
                                         'as_bitter_betty_bought',
                                                     'tty_bought_better_butter_t',
                                                                     'r_butter_to_make_the_',
                                                                                   'ke_the_bitter_butter_better']
sequitur = Sequitur(reads,sequence,assemble=True)
sequitur.sequence == sequence if type(sequitur.sequence) is str else all(s in sequence for s in sequitur.sequence)

True

In [241]:
sequence = 'you say hello world, i bellow go to hell'
reads = ['you say hel',
            ' say hello wo',
                    'lo world, i be',
                          'ld, i bellow go t',
                                    'ow go to hell']
sequitur = Sequitur(reads,sequence,assemble=True)
sequitur.sequence == sequence if type(sequitur.sequence) is str else all(s in sequence for s in sequitur.sequence)

True

In [242]:
sequence = 'she_sells_sea_shells_on_the_sea_shore'
reads = ['she_sells_s',
               'lls_sea_shel',
                    'ea_shells_o',
                       'shells_on_the_s',
                                  'he_sea_s',
                                      'ea_shore']
sequitur = Sequitur(reads,sequence,assemble=True)
sequitur.sequence == sequence if type(sequitur.sequence) is str else all(s in sequence for s in sequitur.sequence)

True

In [245]:
successes = 0
n = 20
for seed in range(n):   
    sequence = generate_genome_sequence(2000,seed=seed)
    reads = generate_reads(sequence,12,20,5,9,seed=seed)
    sequitur = Sequitur(reads,sequence,assemble=True)
    s = '| Seed: ' + str(seed) + ' | '
    if type(sequitur.sequence) is str and sequitur.sequence == sequence:
        s+='SUC | ' + str(sequitur.sequence) + ' == ' + sequence
        successes+=1
    elif type(sequitur.sequence) is list and all(s in sequence for s in sequitur.sequence):
        s+='PAR | ' + str(sequitur.sequence) + ' ~~ ' + sequence
        successes+=0.5
    else: s+='FAI | ' + sequitur.sequence + ' != ' + sequence
    print(s)
    print('-----------------------------------------')
print('ACCURACY: '+str((successes/n)*100)+'%')

| Seed: 0 | PAR | ['TTTAATCAACGAGCTTAATG', 'TTAGTTGTGCCGCAGCGAAGTAGTGCTTGAAATATGCGACCCCTAAGTAGGAGCGTATGCGCCCAGTAACCAATGCCTGTTGAGATGCCAGACGCGTAACCAAAACATAGAAACCATCAATAGACAGGTCATAATCGGTCCACCGGATCATTGGTGCATAGAGCCTGGGCGTTAACGCCCTTTATTACTAGCTTAATG', 'CTTAATGAGCTGACATTGCTGAAATGACCATGACTTAATAATCATTTATGGAGAAGAGGCACGACCACAAGGACCCTATGGCACGGTGGGCAAGCTCCCGCCCGGTACATAACTGTCTGGACTGATTATGTCGGTACAGACTTCTTC', 'AGCTTAATGGTATCACATTGACAAACACGGCATTAAGTAGCGACGAAACGGGATTTGCCTGACCGGGGAGAAGCCGGTCGATCAGCAGTGGTAATTGGATATTAGGCCTAAACCATAATGTTCTAGCGCTCGAAATCATTGCACCACTTGCATCTTTGTTCCAGGGACGCTGTAAAACCAGATGCCTGTAAATCGTTTCAACGGGATGGTTTACCCGGAATTCTACGTATTTAAT', 'CTTCTTCCTAGCGCTCTCTACTGCAAACCTCATGGTCCGGTTTGTACCACTTCGAATCATTCACTGTAATACTGGAACAGCTGATGCTTGCGAACGTATTGTAATCCAATCAGGTAGGGCAGACTAGGCGATAGAACTACGACCCTCTGCGGCCTGCGAAGGGAAAGACGCGTATTAATGCGCTGAAAGAGGGACCGATTGCGGTTATGCACTTACAGACCTAGTACTGGTTTACGCCCCGCCGACCTCCCCCGATCATCTTCTTACAAAACTTGTGTGACCCAAATCTCGACGGGGGCATCTGTGTCTGGGTTTGCCTCGCCTGGTGTACGTAACAGACCCGG', 'ATTTAATCCCCCCACT

KeyboardInterrupt: 

    SUC: returns the target sequence fully reconstructed
    PAR: returns contigs all of which exist in the target sequence (consider coverage?)
    FAI: returns a full sequence that is incorrectly reconstructed or a set of contigs where at least one is not found in the target sequence

In [247]:
seed = 0
sequence = generate_genome_sequence(2000,seed=seed)
reads = generate_reads(sequence,12,20,5,9,seed=seed)
sequitur = Sequitur(reads,sequence,assemble=True)
sequitur.sequence #== sequence if type(sequitur.sequence) is str else all(s in sequence for s in sequitur.sequence)
# would my OG alg struggle with these tandem repeats (i don't recall them being an issue)

['TTTAATCAACGAGCTTAATG',
 'TTAGTTGTGCCGCAGCGAAGTAGTGCTTGAAATATGCGACCCCTAAGTAGGAGCGTATGCGCCCAGTAACCAATGCCTGTTGAGATGCCAGACGCGTAACCAAAACATAGAAACCATCAATAGACAGGTCATAATCGGTCCACCGGATCATTGGTGCATAGAGCCTGGGCGTTAACGCCCTTTATTACTAGCTTAATG',
 'CTTAATGAGCTGACATTGCTGAAATGACCATGACTTAATAATCATTTATGGAGAAGAGGCACGACCACAAGGACCCTATGGCACGGTGGGCAAGCTCCCGCCCGGTACATAACTGTCTGGACTGATTATGTCGGTACAGACTTCTTC',
 'AGCTTAATGGTATCACATTGACAAACACGGCATTAAGTAGCGACGAAACGGGATTTGCCTGACCGGGGAGAAGCCGGTCGATCAGCAGTGGTAATTGGATATTAGGCCTAAACCATAATGTTCTAGCGCTCGAAATCATTGCACCACTTGCATCTTTGTTCCAGGGACGCTGTAAAACCAGATGCCTGTAAATCGTTTCAACGGGATGGTTTACCCGGAATTCTACGTATTTAAT',
 'CTTCTTCCTAGCGCTCTCTACTGCAAACCTCATGGTCCGGTTTGTACCACTTCGAATCATTCACTGTAATACTGGAACAGCTGATGCTTGCGAACGTATTGTAATCCAATCAGGTAGGGCAGACTAGGCGATAGAACTACGACCCTCTGCGGCCTGCGAAGGGAAAGACGCGTATTAATGCGCTGAAAGAGGGACCGATTGCGGTTATGCACTTACAGACCTAGTACTGGTTTACGCCCCGCCGACCTCCCCCGATCATCTTCTTACAAAACTTGTGTGACCCAAATCTCGACGGGGGCATCTGTGTCTGGGTTTGCCTCGCCTGGTGTACGTAACAGACCCGG',
 'ATTTAATCCCCCCACTGGGATGCGGCTGC

In [248]:
any(sequitur.sequence[0] in s for s in sequitur.sequence[1:])

False

# DeBruijn Graph

In [1]:
# ! pip install toyplot networkx
import networkx as nx
import toyplot, math, json

In [15]:
def construct_debruijn_graph(reads,k=3,allow_parallel_edges=True):
    import networkx as nx
    
    if allow_parallel_edges: G = nx.MultiDiGraph()
    else: G = nx.DiGraph()
    for read in reads:
        for i in range(len(read)-k+1):
            G.add_edge(read[i:i+k-1],read[i+1:i+k])
    return G

def plot_debruijn_graph(edges, width=500, height=500):
    "returns a toyplot graph from an input of edges"
    graph = toyplot.graph(
        [i[0] for i in edges],
        [i[1] for i in edges],
        width=width,
        height=height,
        tmarker=">", 
        vsize=25,
        vstyle={"stroke": "black", "stroke-width": 2, "fill": "none"},
        vlstyle={"font-size": "11px"},
        estyle={"stroke": "black", "stroke-width": 2},
        layout=toyplot.layout.FruchtermanReingold(edges=toyplot.layout.CurvedEdges()))
    return graph

def assemble(G):
    seq = ''
    init = True
    if nx.is_eulerian(G) or nx.has_eulerian_path(G):
        for n in nx.eulerian_path(G):
            if init: 
                seq = n[0] + n[1][-1]
                init = False
                continue
            seq += n[1][-1]
        return seq
    else: return False

In [20]:
sequence = 'betty_bought_butter_the_butter_was_bitter_betty_bought_better_butter_to_make_the_bitter_butter_better'
reads = ['betty_bought_butter_th',
                        'tter_the_butter_was_',
                              'he_butter_was_bitter_',
                                         'as_bitter_betty_bought',
                                                     'tty_bought_better_butter_t',
                                                                     'r_butter_to_make_the_',
                                                                                   'ke_the_bitter_butter_better']
for k in range(3,min(len(read) for read in reads)):
      G = construct_debruijn_graph(reads,k)
      seq = assemble(G)
      if not seq: print("| k = " + str(k) + " | FAILURE | **no eulerian path** |")
      elif seq == sequence: print("| k = " + str(k) + " | SUCCESS | " + seq + " |")
      else: print("| k = " + str(k) + " | FAILURE | " + seq + " |")

NetworkXNotImplemented: not implemented for directed type

In [18]:
sequence = 'you say hello world, i bellow go to hell'
reads = ['you say hel',
            ' say hello wo',
                    'lo world, i be',
                          'ld, i bellow go t',
                                    'ow go to hell']
for k in range(3,min(len(read) for read in reads)):
      G = construct_debruijn_graph(reads,k)
      seq = assemble(G)
      if not seq: print("| k = " + str(k) + " | FAILURE | **no eulerian path** |")
      elif seq == sequence: print("| k = " + str(k) + " | SUCCESS | " + seq + " |")
      else: print("| k = " + str(k) + " | FAILURE | " + seq + " |")

| k = 3 | FAILURE | **no eulerian path** |
| k = 4 | FAILURE | **no eulerian path** |
| k = 5 | FAILURE | **no eulerian path** |
| k = 6 | FAILURE | **no eulerian path** |
| k = 7 | FAILURE | **no eulerian path** |
| k = 8 | FAILURE | **no eulerian path** |
| k = 9 | FAILURE | **no eulerian path** |
| k = 10 | FAILURE | **no eulerian path** |


In [19]:
sequence = 'she_sells_sea_shells_on_the_sea_shore'
reads = ['she_sells_s',
               'lls_sea_shel',
                    'ea_shells_o',
                       'shells_on_the_s',
                                  'he_sea_s',
                                      'ea_shore']
for k in range(3,min(len(read) for read in reads)):
      G = construct_debruijn_graph(reads,k)
      seq = assemble(G)
      if not seq: print("| k = " + str(k) + " | FAILURE | **no eulerian path** |")
      elif seq == sequence: print("| k = " + str(k) + " | SUCCESS | " + seq + " |")
      else: print("| k = " + str(k) + " | FAILURE | " + seq + " |")

| k = 3 | FAILURE | **no eulerian path** |
| k = 4 | FAILURE | **no eulerian path** |
| k = 5 | FAILURE | **no eulerian path** |
| k = 6 | FAILURE | **no eulerian path** |
| k = 7 | FAILURE | **no eulerian path** |


In [ ]:
# take gene data and chop it up to represent short and long reads

In [18]:
from Bio import SeqIO

In [24]:
reads = []
for record in SeqIO.parse("data\Batrachochytrium_dendrobatidis\GCA_000203795.1\GCA_000203795.1_v1.0_genomic.fna",'fasta'):
    reads += [record.seq]
reads

[Seq('CATGCAATAAAATAGTCTTTCTGTacttttttcagttgcatcgtttctgattcc...TCC'),
 Seq('ACAATAATCCGACGCAATTACCAATTAATAATTACCCGACGGATTGCTCCGACA...TTC'),
 Seq('TCAGAGAAGACCTGACCCAAAAGAAGTAATCGTGTctatcagtcaatcaacaat...CTG'),
 Seq('GGCGAGAGGCGCGACCCGAGAATATTTACCCCCGACTGTCCCTATTCATCATTA...TAT'),
 Seq('tttttttgcaatttcatgacaatacatgcatgcatcgtacataccaatgcaata...GGA'),
 Seq('ctaatcaaacaactggtcagaatgcaggttaaatactagttaaaatactcgtca...cac'),
 Seq('CGGTTGGATGCCGTGGAAGAAGACACTTGACTGAGGGAGCGTCCTACATAAATC...TAA'),
 Seq('AtaccatgtttggcttgccgCGTCTGAGGAAACGatacccaattgtatcattcc...ATA'),
 Seq('accctaaccctaaccctaaccctaaccctaaccctaaccctaaccctaacccta...ATG'),
 Seq('ATAAAAAAAgagaaaagaaaaaaaggaagaaaagaaaagaaagGATACAAGGAT...TAT'),
 Seq('CACAAATAAACCGGTTACACCATTATCCATCAAAGCTGGGAACCTGTCCATAAT...TTT'),
 Seq('attttattgcaatcatttgtagtCCCATCAGTAACACTCAGGATAGTATCCATA...CGA'),
 Seq('TAACACAACgccaatagcagttatttcccagccatatttaaactgattgtgtat...TCA'),
 Seq('CGATGGTCGAAATAGCGTGTATAATTCAAGTCGCGATGGAAATATTTTTGGCTG...GGA'),
 Seq('Agcctaaccctaac